# CNN


## Simple Convolutional Layer from Scratch

- CNN:
  - convolutional layer
    - convolution computing
    - with padding and stride
  - pooling layer
  - fully-connected layer
  - activate function


In [34]:
def conv(X, K, std=1, pad=0):
    X = X.float()
    
    k_h, k_w = K.shape
    x_h, x_w = X.shape
    
    x_h_pad, x_w_pad = x_h+pad*2, x_w+pad*2 
    X_pad = torch.zeros(x_h_pad,x_w_pad)
    X_pad[pad:pad+x_h, pad:pad+x_w] += X
    
    y_h, y_w = (x_h_pad-k_h)//std + 1, (x_w_pad-k_w)//std + 1
    Y = torch.zeros(y_h,y_w)
    
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X_pad[i*std:i*std+k_h, j*std:j*std+k_w]*K).sum()
    
    return Y

def conv_multi_in(X, K, std=1, pad=0):
    res = conv(X[0,:,:],K[0,:,:],std,pad)
    for i in range(1,X.shape[0]):
        res += conv(X[i,:,:],K[i,:,:],std,pad)
    return res

def conv_multi_in_out(X, K, std=1, pad=0):
    return torch.stack([conv_multi_in(X,k,std,pad) for k in K])

# test
X = torch.tensor([[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15]])
K = torch.tensor([[0, 1], [2, 3]])
print(conv(X, K, 2, 0))   

def pool(X, p_size, std, pad=0, mode='max'):
    X = X.float()
    p_h, p_w = p_size

    x_h, x_w = X.shape
    
    x_h_pad, x_w_pad = x_h+pad*2, x_w+pad*2
    X_pad = torch.zeros(x_h_pad,x_w_pad)
    X_pad[pad:pad+x_h, pad:pad+x_w] += X
    
    y_h, y_w = (x_h_pad-p_h)//std + 1, (x_w_pad-p_w)//std + 1
    Y = torch.zeros(y_h,y_w)
    
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i,j] = X[i*std:i*std+p_h, j*std:j*std+p_w].max()
            elif mode == 'mean':
                Y[i,j] = X[i*std:i*std+p_h, j*std:j*std+p_w].sum()
    return Y

def pool_multi(X, p_size, std, pad=0, mode='max'):
    return torch.cat([pool(x, p_size, std, pad=0, mode='max') for x in X],dim=0)

# test    
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
print(pool(X,(2,2),1,0,'max'))   

  
def sigmoid(X):
    return 1/1+torch.exp(X)

def ReLU(X):
    return torch.max(X,torch.tensor(0.0))
    #return torch.max(input=X,other=torch.tensor(0.0))

def linear(X,W,b):
    return torch.matmul(X,W)+b

def softmax(X):
    return torch.exp(X)/(torch.exp(X).sum(dim=1,keepdim=True))

X = torch.tensor([[0, 0], [3, 4]]).float()
print(softmax(X))

tensor([[24., 36.],
        [72., 84.]])
tensor([[4., 5.],
        [7., 8.]])
tensor([[0.5000, 0.5000],
        [0.2689, 0.7311]])


## CNN with MNIST Using PyTorch


In [32]:
# Import Packages
import torch
from torch import nn as nn
from torch import optim as optim
from torch.utils import data as Data

import torchvision
from torchvision import datasets
from torchvision import transforms

import numpy as np
import pandas as pd 

import d2lzh_pytorch as d2dl


In [17]:
# Hyperparameters
batch_size = 256
num_epochs = 5
learning_rate = 0.01

num_classes = 10

# Load Data
train_data = torchvision.datasets.MNIST(root='/Users/yanzheyuan/coding/dataset_pytorch/',\
                                       train=True, transform=transforms.ToTensor()) 
test_data = torchvision.datasets.MNIST(root='/Users/yanzheyuan/coding/dataset_pytorch/',\
                                       train=False, transform=transforms.ToTensor()) 

train_iterator = Data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_iterator = Data.DataLoader(test_data,batch_size=batch_size,shuffle=True)

# Define Model
class cnn_model(nn.Module):
    def __init__(self,num_classes):
        super(cnn_model,self).__init__()
        self.cnn_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=7,padding=1,stride=1),
            nn.Sigmoid(),
            nn.MaxPool2d(2,2),
            nn.Sigmoid()
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(16*12*12,num_classes)
        )
    def forward(self, x):
        y = self.cnn_layer(x)
        y = self.fc_layer(y.view(x.shape[0],-1))
        return y

net = cnn_model(num_classes=10)
print(net)

loss_func = nn.CrossEntropyLoss()
optimizor = optim.Adam(net.parameters(), lr=learning_rate)

# Train Model
def evaluate_model(net, test_iterator):
    with torch.no_grad():
        correct,num_exp = 0.0,0
        for X,y in test_iterator:
            output = net(X)
            num_exp += y.size(0)
            correct += (output.argmax(1)==y).sum().item()
            
    return correct/num_exp*100

def train_model(num_epochs, train_iterator, test_iterator, loss_func, optimizor, net):
    for epoch in range(num_epochs):
        total_loss,total_batch,total_acc,total_num = 0.0,0,0.0,0
        for X, y in train_iterator:
            output = net(X)
            loss = loss_func(output,y)
            optimizor.zero_grad()
            loss.backward()
            optimizor.step()
            
            total_loss += loss.item()
            total_batch += 1
            total_acc += (output.argmax(1)==y).sum().item()
            total_num += y.size(0)
        
        test_acc = evaluate_model(net, test_iterator)
        print('Epoch: {}, Average loss: {:.4f}, Average accuracy: {:.2f}%, Test Accuracy: {:.2f}%' \
              .format(epoch+1, total_loss/total_batch, total_acc/total_num*100, test_acc))

train_model(num_epochs,train_iterator,test_iterator,loss_func,optimizor,net)
        
# Prediction

cnn_model(
  (cnn_layer): Sequential(
    (0): Conv2d(1, 16, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Sigmoid()
  )
  (fc_layer): Sequential(
    (0): Linear(in_features=2304, out_features=10, bias=True)
  )
)
Epoch: 1, Average loss: 3.6605, Average accuracy: 33.48%, Test Accuracy: 64.36%
Epoch: 2, Average loss: 0.8148, Average accuracy: 76.31%, Test Accuracy: 84.90%
Epoch: 3, Average loss: 0.4729, Average accuracy: 85.77%, Test Accuracy: 81.24%
Epoch: 4, Average loss: 0.3865, Average accuracy: 88.30%, Test Accuracy: 88.59%
Epoch: 5, Average loss: 0.3241, Average accuracy: 90.16%, Test Accuracy: 91.21%
